In [100]:
from Crawler import Crawler

# tool
import pandas as pd
import numpy as np
import tqdm
import pickle
import math
import time
import random
import re
import matplotlib.pylab as plt

## sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import pairwise

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Bert
from transformers import *

In [7]:
class dataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        instance = {}
        sample = self.data.values[index]
        
        instance['id'] = sample[0]
        
        if len(self.data.columns) > 2:
            instance['article'] = sample[2]
            instance['label']   = sample[1]
            
        else:
            nstance['article']  = sample[1]
    
        return instance

    def collate_fn(self, samples):
        
        batch = {}
        
        for key in ['id', 'article']:
            if any(key not in sample for sample in samples):
                continue
            batch[key] = torch.tensor([sample[key] for sample in samples]).float()
            
        for key in ['label']:
            if any(key not in sample for sample in samples):
                continue
            batch[key] = torch.tensor([sample[key] for sample in samples]).view(-1, 1).float()

        return batch

In [10]:
def preprocess(df):
    
    df_copy = df.copy()
    bos_id = '[CLS]'
    
    df_copy['name_split'] = df_copy.article
    
    df_copy['tokens'] = df_copy.article.apply(lambda x: tokenizer.tokenize(x)) ## encode
    df_copy['512_tokens'] = df_copy.tokens.apply(lambda x: [bos_id]+x[:511]) ## split to 512 length
    df_copy['ids'] =  df_copy['512_tokens'].apply(lambda token: tokenizer.convert_tokens_to_ids(token)) ## to ids
        
    return df_copy

In [2]:
crawler = Crawler()

In [3]:
train_csv = pd.read_csv('train.csv')

In [86]:
train_csv_no_404 = train_csv[train_csv.article != '文章已被刪除 404 or 例外']\
.drop(['hyperlink', 'domain'], axis=1)

In [87]:
train_csv_no_404.head(5)

,news_ID,content,name,article
0,1,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[],理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:3...
1,2,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[],10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...
2,3,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[],社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從...
4,5,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...
5,6,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[],內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（註...


## Preprocess

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [13]:
train_csv_no_404['label'] = train_csv_no_404.name.apply(lambda x: 0 if len(x) == 2 else 1)
train_csv_no_404 = train_csv_no_404.reset_index(drop = True)

In [14]:
train_df = preprocess(train_csv_no_404)

In [15]:
train_df[train_df.name != '[]']

,news_ID,name,article,label,tokens,512_tokens,ids
15,18,['王派宏'],自稱房產幽默大師的王派宏，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控...,1,"[自, 稱, 房, 產, 幽, 默, 大, 師, 的, 王, 派, 宏, ，, 涉, 吸, ...","[[CLS], 自, 稱, 房, 產, 幽, 默, 大, 師, 的, 王, 派, 宏, ，,...","[101, 5632, 4935, 2791, 4496, 2406, 7949, 1920..."
35,38,"['王桂霜', '李威儀', '藍秀琪']",〔記者張文川／台北報導〕12年前爆發的花蓮縣壽豐鄉鯉魚潭風景區的「綠湖國際大飯店」開發弊案，...,1,"[〔, 記, 者, 張, 文, 川, ／, 台, 北, 報, 導, 〕, 12, 年, 前,...","[[CLS], 〔, 記, 者, 張, 文, 川, ／, 台, 北, 報, 導, 〕, 12...","[101, 526, 6250, 5442, 2484, 3152, 2335, 8027,..."
38,41,"['陳鏡如', '陳星佑']",〔記者楊國文／台北報導〕「台灣第一家」有限公司，被查出將砷含量超標的工業用碳酸鎂摻入胡椒粉、...,1,"[〔, 記, 者, 楊, 國, 文, ／, 台, 北, 報, 導, 〕, 「, 台, 灣, ...","[[CLS], 〔, 記, 者, 楊, 國, 文, ／, 台, 北, 報, 導, 〕, 「,...","[101, 526, 6250, 5442, 3501, 1751, 3152, 8027,..."
63,66,['朱小蓉'],苗栗一群外配向本報投訴〔記者鄭名翔／苗栗報導〕苗栗縣一群外籍配偶向本報投訴，指控從越南嫁來台...,1,"[苗, 栗, 一, 群, 外, 配, 向, 本, 報, 投, 訴, 〔, 記, 者, 鄭, ...","[[CLS], 苗, 栗, 一, 群, 外, 配, 向, 本, 報, 投, 訴, 〔, 記,...","[101, 5728, 3412, 671, 5408, 1912, 6981, 1403,..."
70,73,['廖泰宇'],捲入馬勝金融集團（MaximTrader）吸金案，今年8月才被新北地院依違反銀行法判刑8年的...,1,"[捲, 入, 馬, 勝, 金, 融, 集, 團, （, [UNK], ）, 吸, 金, 案,...","[[CLS], 捲, 入, 馬, 勝, 金, 融, 集, 團, （, [UNK], ）, 吸...","[101, 2947, 1057, 7679, 1245, 7032, 6084, 7415..."
...,...,...,...,...,...,...,...
4811,4953,"['王隆昌', '吳淑珍']",▲前大法官許玉秀（左）為王隆昌（麥克風者）聲援背書，但仍不被最高法院接受，駁回再審聲請。（圖...,1,"[▲, 前, 大, 法, 官, 許, 玉, 秀, （, 左, ）, 為, 王, 隆, 昌, ...","[[CLS], ▲, 前, 大, 法, 官, 許, 玉, 秀, （, 左, ）, 為, 王,...","[101, 464, 1184, 1920, 3791, 2135, 6258, 4373,..."
4830,4972,"['王姝茵', '張陳淑媜', '張安樂', '張瑋', '李新一', '張馥堂']",遭控收受政治獻金 未按規定申報〔記者黃捷／台北報導〕中華統一促進黨總裁「白狼」張安樂及其子張...,1,"[遭, 控, 收, 受, 政, 治, 獻, 金, 未, 按, 規, 定, 申, 報, 〔, ...","[[CLS], 遭, 控, 收, 受, 政, 治, 獻, 金, 未, 按, 規, 定, 申,...","[101, 6901, 2971, 3119, 1358, 3124, 3780, 4368..."
4838,4980,['楚瑞芳'],〔記者錢利忠／台北報導〕曾被週刊爆料涉嫌假冒華固建設高層，並涉土地投資詐騙糾紛的前國防部政治...,1,"[〔, 記, 者, 錢, 利, 忠, ／, 台, 北, 報, 導, 〕, 曾, 被, 週, ...","[[CLS], 〔, 記, 者, 錢, 利, 忠, ／, 台, 北, 報, 導, 〕, 曾,...","[101, 526, 6250, 5442, 7092, 1164, 2566, 8027,..."
4839,4981,['邱佳亮'],桃園市議員邱佳亮被控貪污案，一審遭判12年6月，經上訴由二審審理中，邱以市議會受邀，將到上海...,1,"[桃, 園, 市, 議, 員, 邱, 佳, 亮, 被, 控, 貪, 污, 案, ，, 一, ...","[[CLS], 桃, 園, 市, 議, 員, 邱, 佳, 亮, 被, 控, 貪, 污, 案,...","[101, 3425, 1754, 2356, 6359, 1519, 6937, 881,..."


## Split Data

In [11]:
def split_data(df):
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    
    for train_index, test_index in skf.split(df, df['label']):
        X_train, tempt = df.loc[train_index], df.loc[test_index]

    tempt = tempt.reset_index(drop = True)
    
    for test_index, val_index in skf.split(tempt, tempt['label']):
        X_test, X_val = tempt.loc[test_index], tempt.loc[val_index]
        
    return X_train, X_test, X_val

In [12]:
X_train, X_test, X_val = split_data(train_df)

In [13]:
train_dataset = dataset(X_train)
val_dataset   = dataset(X_test)
test_dataset  = dataset(X_val)

n_train = len(train_dataset)
n_val = len(val_dataset)
n_test = len(test_dataset)